Experimentation on the Syntetic simmulated data and annotators
* batch = 128
* delta convergence = 1e-2
* Optimizer = ADAM

* Our proposed: Pre-train base model with hard-MV (5 epochs?) as Rodrigues: https://github.com/fmpr/CrowdLayer/blob/master/demo-conll-ner-mturk.ipynb

In [136]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras, time, sys, os, gc
from keras.models import clone_model

DTYPE_OP = 'float32'
keras.backend.set_floatx(DTYPE_OP)

if DTYPE_OP == 'float64':
    keras.backend.set_epsilon(np.finfo(np.float64).eps)
elif DTYPE_OP == 'float32':
    keras.backend.set_epsilon(np.finfo(np.float32).eps)

In [2]:
#GLOBAL Variables
BATCH_SIZE = 128
EPOCHS_BASE = 50
OPT = 'adam' #optimizer for neural network 
TOL = 1e-2 #tolerance for relative variation of parameters

### Load Data and preprocess

In [3]:
folder = ".." #same as refered below..

X_train = np.loadtxt(folder+"/syntetic/simple/datasim_X_train.csv",delimiter=',')
Z_train = np.loadtxt(folder+"/syntetic/simple/datasim_Z_train.csv",dtype='int') #groudn truth

X_test = np.loadtxt(folder+"/syntetic/simple/datasim_X_test.csv",delimiter=',')
Z_test = np.loadtxt(folder+"/syntetic/simple/datasim_Z_test.csv",dtype='int') #groudn truth

print("Input shape:",X_train.shape)

Input shape: (2292, 2)


In [4]:
from sklearn.preprocessing import StandardScaler
std= StandardScaler(with_mean=True) #matrices sparse with_mean=False
std.fit(X_train)
Xstd_train = std.transform(X_train)
Xstd_test = std.transform(X_test)
Xstd_train.shape

(2292, 2)

#### Delta Convergence criteria

In [19]:
from code.learning_models import LogisticRegression_Sklearn,LogisticRegression_Keras,MLP_Keras
#deep learning
from code.learning_models import default_CNN,default_RNN,default_RNNw_emb,CNN_simple, RNN_simple

from code.utils import EarlyStopRelative
ourCallback = EarlyStopRelative(monitor='loss',patience=1,min_delta=TOL)

In [20]:
#upper bound model
Z_train_onehot = keras.utils.to_categorical(Z_train)

model_UB = MLP_Keras(Xstd_train.shape[1:],Z_train_onehot.shape[1],8,1,BN=False,drop=0.2)

model_UB.compile(loss='categorical_crossentropy',optimizer=OPT)
model_UB.fit(Xstd_train,Z_train_onehot,epochs=EPOCHS_BASE,batch_size=BATCH_SIZE,verbose=0,callbacks=[ourCallback])

evaluate = Evaluation_metrics(model_UB,'keras',Xstd_train.shape[0],plot=False)
Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred)
Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred)

results1[0].to_csv("synthetic_UpperBound_train.csv",index=False)
results2[0].to_csv("synthetic_UpperBound_test.csv",index=False)

In [131]:
def get_mean_dataframes(df_values):
    if df_values[0].iloc[:,0].dtype == object:
        RT = pd.DataFrame(data=None,columns = df_values[0].columns[1:], index= df_values[0].index)
    else:
        RT = pd.DataFrame(data=None,columns = df_values[0].columns, index= df_values[0].index)
        
    data = []
    for df_value in df_values:
        if df_value.iloc[:,0].dtype == object:
            data.append( df_value.iloc[:,1:].values )
        else:
            data.append(df_value.values)
    RT[:] = np.mean(data,axis=0)
    
    if df_values[0].iloc[:,0].dtype == object:
        RT.insert(0, "", df_values[0].iloc[:,0].values )
    return RT

In [108]:
from code.representation import *
from code.utils import *

In [109]:
from code.baseline import LabelInference
from code.baseline import RaykarMC
from code.MixtureofGroups import GroupMixtureOpt
from code.MixtureofGroups import project_and_cluster,clusterize_annotators
from code.evaluation import Evaluation_metrics

In [110]:
M_seted = 3 #arg

### Generate Annotations

In [111]:
from code.generate_data import SinteticData

In [14]:
GenerateData = SinteticData()
GenerateData.set_probas(asfile=True,file_matrix='../syntetic/simple/matrix_datasim_normal.csv',file_groups ='../syntetic/simple/groups_datasim.csv')
Tmax = 100
T_data = 10 
y_obs, groups_annot = GenerateData.sintetic_annotate_data(Z_train,Tmax,T_data,deterministic=False)


real_conf_matrix = GenerateData.conf_matrix

if len(groups_annot.shape) ==1 or groups_annot.shape[1] ==  1: 
    groups_annot = keras.utils.to_categorical(groups_annot)  #only if it is hard clustering
    print("Done")
confe_matrix = np.tensordot(groups_annot,real_conf_matrix, axes=[[1],[0]])

del GenerateData
gc.collect()

Done


159

In [133]:
scenario = 1  #arg

In [143]:
folder = ".." #folder of where is syntetic folder --arg

GenerateData = SinteticData()

#CONFUSION MATRIX CHOOSE
if scenario == 1 or scenario == 3 or scenario == 4 or scenario == 5 or scenario == 6:
    GenerateData.set_probas(asfile=True,file_matrix=folder+'/syntetic/simple/matrix_datasim_normal.csv',file_groups =folder+'/syntetic/simple/groups_datasim.csv')

elif scenario == 2 or scenario == 7: #bad MV
    GenerateData.set_probas(asfile=True,file_matrix=folder+'/syntetic/simple/matrix_datasim_badMV.csv',file_groups =folder+'/syntetic/simple/groups_datasim.csv')

real_conf_matrix = GenerateData.conf_matrix.copy()


#ANNOTATOR DENSITY CHOOSE
if scenario == 1 or scenario ==2 or scneario == 3:
    Tmax = 100
    T_data = 10 
    
elif scenario == 4 or scenario == 7:
    Tmax = 2000
    T_data = 20 
    
elif scenario == 5:
    Tmax = 5000
    T_data = 25

elif scenario == 6:
    Tmax = 10000
    T_data = 40


results_softmv_train = []
results_softmv_test = []
results_hardmv_train = []
results_hardmv_test = []
results_ds_train = []
results_ds_test = []
results_raykar_train = []
results_raykar_trainA = []
results_raykar_test = []
results_ours1_train = []
results_ours1_trainA = []
results_ours1_test = []
results_ours1_testA = []
results_ours2_train = []
results_ours2_trainA = []
results_ours2_test = []
results_ours2_testA = []
results_ours3_train = []
results_ours3_trainA = []
results_ours3_test = []
results_ours3_testA = []

for _ in range(5): #repetitions
    print("New Synthetic data is being generated...",flush=True,end='')
    if scenario == 3: #soft
        y_obs, groups_annot = GenerateData.sintetic_annotate_data(Z_train,Tmax,T_data,deterministic=False,hard=False)
    else:
        y_obs, groups_annot = GenerateData.sintetic_annotate_data(Z_train,Tmax,T_data,deterministic=False)
    print("Done! ")
    
    if len(groups_annot.shape) ==1 or groups_annot.shape[1] ==  1: 
        groups_annot = keras.utils.to_categorical(groups_annot)  #only if it is hard clustering
    confe_matrix = np.tensordot(groups_annot,real_conf_matrix, axes=[[1],[0]])

    N,T = y_obs.shape
    K = np.max(y_obs)+1 # asumiendo que estan ordenadas
    print("Shape (data,annotators): ",(N,T))
    print("Classes: ",K)
    
    ############# EXECUTE ALGORITHMS #############################

    label_I = LabelInference(y_obs,TOL,type_inf = 'all')  #Infer Labels

    mv_onehot = label_I.mv_labels('onehot')
    mv_probas = label_I.mv_labels('probas')

    ds_labels = label_I.DS_labels()
    
    print("ACC MV on train:",np.mean(mv_onehot.argmax(axis=1)==Z_train))
    print("ACC D&S on train:",np.mean(ds_labels.argmax(axis=1)==Z_train))
        
    model_mvsoft = clone_model(model_UB) 
    model_mvsoft.compile(loss='categorical_crossentropy',optimizer=OPT)
    model_mvsoft.fit(Xstd_train, mv_probas, epochs=EPOCHS_BASE,batch_size=BATCH_SIZE,verbose=0)
    print("Trained model over soft-MV")

    model_mvhard = clone_model(model_UB) 
    model_mvhard.compile(loss='categorical_crossentropy',optimizer=OPT)
    model_mvhard.fit(Xstd_train, mv_onehot, epochs=EPOCHS_BASE,batch_size=BATCH_SIZE,verbose=0)
    print("Trained model over hard-MV")

    model_ds = clone_model(model_UB) 
    model_ds.compile(loss='categorical_crossentropy',optimizer=OPT)
    model_ds.fit(Xstd_train, ds_labels, epochs=EPOCHS_BASE,batch_size=BATCH_SIZE,verbose=0)
    print("Trained model over D&S")

    #get representation needed for Raykar
    #y_obs_categorical = label_I.y_obs_categ
    y_obs_categorical = set_representation(y_obs,'onehot') 
    
    raykarMC = RaykarMC(Xstd_train.shape[1:],y_obs_categorical.shape[-1],T,epochs=1,optimizer=OPT,DTYPE_OP=DTYPE_OP)
    raykarMC.define_model('mlp',8,1,BatchN=False,drop=0.2)
    logL_hist = raykarMC.stable_train(Xstd_train,y_obs_categorical,batch_size=BATCH_SIZE,max_iter=EPOCHS_BASE,tolerance=TOL)
    print("Trained model over Raykar")

    #get our representation 
    r_obs = set_representation(y_obs_categorical,"repeat")
    print("vector of repeats:\n",r_obs)
    print("shape:",r_obs.shape)
    
    #pre analysis
    M_ffff,annotators_pca = project_and_cluster(y_obs_categorical,return_projected=True,DTYPE_OP=DTYPE_OP)
    print("Annotators PCA of annotations shape: ",annotators_pca.shape)

    aux = [entropy(example)/np.log(r_obs.shape[1]) for example in mv_probas]
    print("Normalized entropy (0-1) of repeats annotations:",np.mean(aux))
    
    gMixture1 = GroupMixtureOpt(Xstd_train.shape[1:],Kl=r_obs.shape[1],M=M_seted,epochs=1,pre_init=5,optimizer=OPT,dtype_op=DTYPE_OP) 
    gMixture1.define_model("mlp",8,1,BatchN=False,drop=0.2)
    gMixture1.lambda_random = False #lambda=1     
    logL_hists,i  = gMixture1.multiples_run(1,Xstd_train,r_obs,batch_size=BATCH_SIZE,max_iter=EPOCHS_BASE,tolerance=TOL*2
                                       ,cluster=True,bulk_annotators=[y_obs_categorical,annotators_pca])
    print("Trained model over Ours (1)")
    
    gMixture2 = GroupMixtureOpt(Xstd_train.shape[1:],Kl=r_obs.shape[1],M=M_seted,epochs=1,pre_init=5,optimizer=OPT,dtype_op=DTYPE_OP) 
    gMixture2.define_model("mlp",8,1,BatchN=False,drop=0.2)
    gMixture2.lambda_random = True #lambda random
    logL_hists,i = gMixture2.multiples_run(1,Xstd_train,r_obs,batch_size=BATCH_SIZE,max_iter=EPOCHS_BASE,tolerance=TOL*2
                                       ,cluster=True,bulk_annotators=[y_obs_categorical,annotators_pca])
    print("Trained model over Ours (2)")
    
    gMixture3 = GroupMixtureOpt(Xstd_train.shape[1:],Kl=r_obs.shape[1],M=M_seted,epochs=1,pre_init=5,optimizer=OPT,dtype_op=DTYPE_OP) 
    gMixture3.define_model("mlp",8,1,BatchN=False,drop=0.2)
    gMixture3.lambda_random = True #with lambda random --necessary
    logL_hists,i = gMixture3.multiples_run(1,Xstd_train,r_obs,batch_size=BATCH_SIZE,max_iter=EPOCHS_BASE,tolerance=TOL*2
                                   ,cluster=True)
    print("Trained model over Ours (3)")

    
    ################## MEASURE PERFORMANCE ##################################
    
    evaluate = Evaluation_metrics(model_mvsoft,'keras',Xstd_train.shape[0],plot=False)
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred)
    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred)
    
    results_softmv_train += results1
    results_softmv_test += results2

    evaluate = Evaluation_metrics(model_mvhard,'keras',Xstd_train.shape[0],plot=False)
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred)
    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred)
    
    results_hardmv_train += results1
    results_hardmv_test += results2

    evaluate = Evaluation_metrics(model_ds,'keras',Xstd_train.shape[0],plot=False)
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred)
    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred)
    
    results_ds_train += results1
    results_ds_test += results2
    
    evaluate = Evaluation_metrics(raykarMC,'raykar',plot=False)
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    prob_Yzt = raykarMC.get_confusionM()
    prob_Yxt = raykarMC.get_predictions_annot(Xstd_train)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred,conf_pred=prob_Yzt,conf_true=confe_matrix,y_o=y_obs,yo_pred=prob_Yxt)

    results1_aux = evaluate.calculate_metrics(y_o=y_obs,yo_pred=prob_Yxt)

    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred)
    
    results_raykar_train += results1
    results_raykar_trainA += results1_aux
    results_raykar_test += results2

    evaluate = Evaluation_metrics(gMixture1,'our1',plot=False) 
    aux = gMixture1.calculate_extra_components(Xstd_train,y_obs,T=T,calculate_pred_annotator=True)
    predictions_m,prob_Gt,prob_Yzt,prob_Yxt =  aux #to evaluate...
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    y_o_groups = predictions_m.argmax(axis=-1)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred,conf_pred=prob_Yzt,conf_true=confe_matrix,y_o=y_obs,yo_pred=prob_Yxt, y_o_groups=y_o_groups)

    results1_aux = evaluate.calculate_metrics(y_o=y_obs,yo_pred=prob_Yxt)

    c_M = gMixture1.get_confusionM()
    y_o_groups = gMixture1.get_predictions_groups(Xstd_test).argmax(axis=-1) #obtain p(y^o|x,g=m) and then argmax
    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred,conf_pred=c_M, y_o_groups=y_o_groups)
    
    results_ours1_train  += results1
    results_ours1_trainA += results1_aux
    results_ours1_testA.append(results2[0])
    results_ours1_test.append(results2[1])

    evaluate = Evaluation_metrics(gMixture2,'our1',plot=False) 
    aux = gMixture2.calculate_extra_components(Xstd_train,y_obs,T=T,calculate_pred_annotator=True)
    predictions_m,prob_Gt,prob_Yzt,prob_Yxt =  aux #to evaluate...
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    y_o_groups = predictions_m.argmax(axis=-1)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred,conf_pred=prob_Yzt,conf_true=confe_matrix,y_o=y_obs,yo_pred=prob_Yxt, y_o_groups=y_o_groups)

    results1_aux = evaluate.calculate_metrics(y_o=y_obs,yo_pred=prob_Yxt)

    c_M = gMixture2.get_confusionM()
    y_o_groups = gMixture2.get_predictions_groups(Xstd_test).argmax(axis=-1) #obtain p(y^o|x,g=m) and then argmax
    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred,conf_pred=c_M, y_o_groups=y_o_groups)
    
    results_ours2_train += results1
    results_ours2_trainA += results1_aux
    results_ours2_testA.append(results2[0])
    results_ours2_test.append(results2[1])

    evaluate = Evaluation_metrics(gMixture3,'our1',plot=False) 
    aux = gMixture3.calculate_extra_components(Xstd_train,y_obs,T=T,calculate_pred_annotator=True)
    predictions_m,prob_Gt,prob_Yzt,prob_Yxt =  aux #to evaluate...
    Z_train_pred = evaluate.tested_model.predict_classes(Xstd_train)
    y_o_groups = predictions_m.argmax(axis=-1)
    results1 = evaluate.calculate_metrics(Z=Z_train,Z_pred=Z_train_pred,conf_pred=prob_Yzt,conf_true=confe_matrix,y_o=y_obs,yo_pred=prob_Yxt, y_o_groups=y_o_groups)

    results1_aux = evaluate.calculate_metrics(y_o=y_obs,yo_pred=prob_Yxt)

    c_M = gMixture3.get_confusionM()
    y_o_groups = gMixture3.get_predictions_groups(Xstd_test).argmax(axis=-1) #obtain p(y^o|x,g=m) and then argmax
    Z_test_pred = evaluate.tested_model.predict_classes(Xstd_test)
    results2 = evaluate.calculate_metrics(Z=Z_test,Z_pred=Z_test_pred,conf_pred=c_M, y_o_groups=y_o_groups)

    results_ours3_train +=  results1
    results_ours3_trainA += results1_aux
    results_ours3_testA.append(results2[0])
    results_ours3_test.append(results2[1])
    
    print("All Performance Measured")
    
del GenerateData
gc.collect()

#plot measures    
get_mean_dataframes(results_softmv_train).to_csv("synthetic_softMV_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_softmv_test).to_csv("synthetic_softMV_test_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_hardmv_train).to_csv("synthetic_hardMV_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_hardmv_test).to_csv("synthetic_hardMV_test_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_ds_train).to_csv("synthetic_DS_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ds_test).to_csv("synthetic_DS_test_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_ds_train).to_csv("synthetic_DS_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ds_test).to_csv("synthetic_DS_test_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_raykar_train).to_csv("synthetic_Raykar_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_raykar_trainA).to_csv("synthetic_Raykar_trainAnn_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_raykar_test).to_csv("synthetic_Raykar_test_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_ours1_train).to_csv("synthetic_Ours1_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours1_trainA).to_csv("synthetic_Ours1_trainAnn_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours1_test).to_csv("synthetic_Ours1_test_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours1_testA).to_csv("synthetic_Ours1_testAux_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_ours2_train).to_csv("synthetic_Ours2_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours2_trainA).to_csv("synthetic_Ours2_trainAnn_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours2_test).to_csv("synthetic_Ours2_test_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours2_testA).to_csv("synthetic_Ours2_testAux_s"+str(scenario)+".csv",index=False)

get_mean_dataframes(results_ours3_train).to_csv("synthetic_Ours3_train_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours3_trainA).to_csv("synthetic_Ours3_trainAnn_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours3_test).to_csv("synthetic_Ours3_test_s"+str(scenario)+".csv",index=False)
get_mean_dataframes(results_ours3_testA).to_csv("synthetic_Ours3_testAux_s"+str(scenario)+".csv",index=False)

New Synthetic data is being generated...Done! 
Shape (data,annotators):  (2292, 100)
Classes:  3
num Patients: 2292
Iter	log-likelihood	delta-CM	delta-ER	delta-LL
1 	 -15413.840275687367
2 	 -11978.084033037498 	0.5473	0.387760	0.222901
3 	 -11602.744457754245 	0.0827	0.283311	0.031336
4 	 -11592.025886143563 	0.0092	0.168930	0.000924
5 	 -11591.071668088716 	0.0017	0.054112	0.000082
6 	 -11590.797826513093 	0.0005	0.020210	0.000024
7 	 -11590.688835593966 	0.0002	0.009198	0.000009
Class marginals
[0.33 0.33 0.33]
Patient classes
1 [0. 0. 1.]
2 [1. 0. 0.]
3 [0. 1. 0.]
4 [0. 1. 0.]
5 [0. 0. 1.]
6 [0. 0. 1.]
7 [0. 0. 1.]
8 [1. 0. 0.]
9 [1. 0. 0.]
10 [1. 0. 0.]
ACC MV on train: 0.7552356020942408
ACC D&S on train: 0.9917102966841187
Trained model over soft-MV
Trained model over hard-MV
Trained model over D&S
Needed params (units,deep,drop,BatchN?)
Initializing new EM...
Betas shape:  (100, 3, 3)
Q estimate shape:  (2292, 3)
Iter 1/50 
M step: done,  E step: done //  (in 1.00 sec)	logL: -1

In [144]:
i

0

In [141]:
results_softmv_train[0]

[           Accuracy  F1 (micro)
 0  Global  0.777923    0.777923]

In [7]:
#delete unlabeled data
mask_labeled = np.sum(y_obs != -1,axis=1) != 0
Xstd_train = Xstd_train[mask_labeled]
Z_train = Z_train[mask_labeled]
y_obs = y_obs[mask_labeled]
print("Shape (data,annotators): ",(y_obs.shape))

Shape (data,annotators):  (2292, 10000)
